In [2]:
import flowio
import numpy as np
import pandas as pd

In [4]:
import os

In [5]:
file_path = 'OriginalDATA/FlowRepository_FR-FCM-Z24N_files'
files = os.listdir(file_path)

In [6]:
files.sort()

In [7]:
controls = [f for f in files if "KTR" in f]

In [94]:
unstim_control = [f for f in controls if 'unstim' in f]
stim_control = [f for f in controls if ' stim' in f]

In [107]:
ras = [f for f in files if "RA" in f]

In [90]:
unstim_ras = [f for f in ras if "unstim" in f]
stim_ras = [f for f in ras if " stim" in f]

In [10]:
p = "OriginalDATA/FlowRepository_FR-FCM-Z24N_files/10_A_KTR-019 stim_Ungated.fcs"
p2 = "OriginalDATA/FlowRepository_FR-FCM-Z24N_files/11_D_RA-091 unstim_Time.fcs"

In [109]:
import re

In [119]:
re.findall("(RA-[0-9]+)", ras[0])

['RA-072']

In [150]:
unstim_control

['1_A_KTR-029 unstim_Ungated.fcs',
 '1_B_KTR-017 unstim_Time.fcs',
 '1_C_KTR-020 unstim_Time.fcs',
 '1_D_KTR-006 unstim_Time.fcs',
 '3_A_KTR-024 unstim_Ungated.fcs',
 '3_B_KTR-003 unstim_Time.fcs',
 '3_C_KTR-014 unstim_Time.fcs',
 '3_D_KTR-004 unstim_Time.fcs',
 '5_A_KTR-015 unstim_Ungated.fcs',
 '5_B_KTR-021 unstim_Time.fcs',
 '5_C_KTR-007 unstim_Time.fcs',
 '5_D_KTR-028 unstim_Time.fcs',
 '7_A_KTR-023 unstim_Ungated.fcs',
 '7_B_KTR-022 unstim_Time.fcs',
 '7_C_KTR-025 unstim_Time.fcs',
 '7_D_KTR-030 unstim_Time.fcs',
 '9_A_KTR-019 unstim_Ungated.fcs',
 '9_B_KTR-016 unstim_Time.fcs',
 '9_C_KTR-018 unstim_Time.fcs',
 '9_D_KTR-008 unstim_Time.fcs']

### TODO
- get a list of the channels that I want with their channel number
- from the pandas get this channels
- name the channels appropriately

### Note
- Cannot find functional marker p-Erk

# RA-data description

Mass cytometry (CyTOF)
1. panel of 12 phenotyping and 10 functional markers
2. signaling in unstimulated and TNF-stimulated peripheral blood mononulear cells
3. from 20 newly diagnosed, untreated RA patients
4. 20 healthy donors.

1. unstim = unstimulated with tumor necrosis factor (TNF) (TNF inhibitors effectively repress inflammatory activity in RA)
2. stim = stimulated with TNF
3. ungated = manual removal of events representing 

## The RA paper:
The objective of this study was to identify markers in immune cell populations that distinguish RA patients from healthy donors with an emphasis on TNF signaling

We employed mass cytometry (CyTOF) with a panel of 13 phenotyping and 10 functional markers to explore signaling in unstimulated and TNF-stimulated peripheral blood mononuclear cells from 20 newly diagnosed, untreated RA patients and 20 healthy donors.

1. intracellular functional markers
2. surface phenotypic markers 

problem:
- want to collect more cells for each patient
    - don't know what file belongs to each patient
    - don't know which of the data is preprosessed or not
- want to use more markers <b style="color:green"> DONE </b> 
    - don't know how to get the other markers 
    - don't know which markers are phenotypical and which are functional
    

In [136]:
def get_events(file, markers, nr_events):
    f = flowio.flowdata.FlowData(file)
    marker_channels = dict()
    for j in markers:
        for k in f.channels.keys():
            if (j in f.channels[k]['PnS']):
                marker_channels[int(k)] = f.channels[k]['PnS']
    
    npy_data = np.reshape(f.events, (-1, f.channel_count))[:nr_events,:]
    df = pd.DataFrame(npy_data)
    df2 = df.rename(columns=marker_channels)
    return df2[list(marker_channels.values())]

In [79]:
# Taken from the paper
phenotyping = ['CD20','CD3','CD4','CD8a','CD45RA', 'CD56', 'CD56', 
               'CD16', 'CD14', 'CD61', 'CD11c','CD123', 'HLA-DR', 'CD45']

functional = ['Caspase3', 'CD86','p-p38','p-Erk','p-Akt','p-cJun','p-NFkB','IkBa','CD120a','CD120b']

In [ ]:
# choose files
# choose nr of cells per patient
# concatenate all files into one df
# save the file
# write about the data

In [ ]:
"{id_ket}"

In [104]:
nr_cells = 20000

In [151]:
def get_patients(group_files, nr_cells, file_path, markers, group_name, id_key):
    all_df = None
    for i, file in enumerate(group_files):
        path = file_path + "/" + file
        df = get_events(path, markers, nr_cells)
        key = "({}-[0-9]+)".format(id_key)
        df["id"] = i #re.findall(key, file)[0]
        if i == 0:
            all_df = df
        else:
            all_df = all_df.append(df)
    all_df["group"]=group_name
    return all_df

In [154]:
df = get_patients(unstim_control, 100, file_path, phenotyping, "control", "KTR")

In [155]:
np.unique(df.id.values)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19])

In [158]:
df.columns

Index(['147Sm_CD20', '170Er_CD3', '89Y_CD45', '145Nd_CD4', '169Tm_CD45RA',
       '146Nd_CD8a', '176Yb_CD56', '148Nd_CD16', '160Gd_CD14', '209Bi_CD61',
       '159Tb_CD11c', '151Eu_CD123', '174Yb_HLA-DR', 'id', 'group'],
      dtype='object')